In [2]:
import numpy as np
import pandas as pd
import tensorflow as tf
import matplotlib.pyplot as plt

In [3]:
train = pd.read_csv('mnist_train.csv')
test = pd.read_csv('mnist_test.csv')

In [4]:
dict_x_train = train.ix[:,'0':]
pixel_train = dict_x_train.as_matrix()
dict_y_train = train.ix[:,'5']
label = dict_y_train.as_matrix()

dict_x_test = test.ix[:,'0':]
pixel_test = dict_x_test.as_matrix()
dict_y_test = test.ix[:,'7']
label_test = dict_y_test.as_matrix()

In [5]:
mean_train = np.mean(pixel_train,axis = 1)
std_train = np.std(pixel_train,axis = 1)

mean_test = np.mean(pixel_test,axis = 1)
std_test = np.std(pixel_test,axis = 1)

In [6]:
mean_temp_train = np.tile(mean_train,[784,1])
mean_temp_train = mean_temp_train.transpose()

std_temp_train = np.tile(std_train,[784,1])
std_temp_train = std_temp_train.transpose()


mean_temp_test = np.tile(mean_test,[784,1])
mean_temp_test = mean_temp_test.transpose()

std_temp_test = np.tile(std_test,[784,1])
std_temp_test = std_temp_test.transpose()

In [7]:
zero_mean_train = pixel_train - mean_temp_train
data = zero_mean_train/std_temp_train


zero_mean_test = pixel_test - mean_temp_test
test_data = zero_mean_test/std_temp_test

In [8]:
train_data = data

In [91]:
data = train_data[0:42000,:]
validate = train_data[42001:54000,:]
testdata = train_data[54001:59998,:]

label_data = label[0:42000]
label_validate = label[42001:54000]
label_test = label[54001:59998]


In [10]:
def one_hot_encoding(x):
    temp = np.zeros((len(x), 10))             #10 classes
    temp[np.arange(len(x)), x] = 1
    return temp

In [11]:
image_size = 28

image_size_flat = image_size*image_size

image_shape = (image_size,image_size)

num_channels = 1

num_classes = 10

In [12]:
def new_weights(shape):
    return tf.Variable(tf.truncated_normal(shape))

def new_biases(length):
    return tf.Variable(tf.constant(0.0, shape=[length]))

In [13]:
def flatten_layer(layer):

    layer_shape = layer.get_shape()

    num_features = layer_shape[1:4].num_elements()
    
    layer_flat = tf.reshape(layer, [-1, num_features])

    return layer_flat, num_features

In [14]:
x = tf.placeholder(tf.float32, shape=[None, image_size_flat], name='x')
x_image = tf.reshape(x, [-1, image_size, image_size, num_channels])
y_true = tf.placeholder(tf.float32, [None, num_classes])
y_true_cls = tf.argmax(y_true, dimension=1)

In [15]:
filter_size1 = 5
num_filters1 = 6 

In [16]:
shape_1 = [filter_size1, filter_size1, num_channels, num_filters1]

first_weights = new_weights(shape = shape_1)

first_biases = new_biases(length=num_filters1)

first_layer = tf.nn.conv2d(input=x_image, ################################################################# conv2d
                         filter=first_weights,
                         strides=[1, 1, 1, 1],
                         padding='SAME')

first_layer += first_biases

first_layer = tf.nn.max_pool(value=first_layer,
                               ksize=[1, 2, 2, 1],
                               strides=[1, 2, 2, 1],
                               padding='SAME')
first_layer = tf.nn.relu(first_layer)

In [17]:
filter_size2 = 5        
num_filters2 = 36  

In [18]:
shape_2 = [filter_size2, filter_size2, num_filters1, num_filters2]

second_weights = new_weights(shape=shape_2)

second_biases = new_biases(length=num_filters2)

second_layer = tf.nn.conv2d(input=first_layer, ################################################################# conv2d
                    filter=second_weights,
                    strides=[1, 1, 1, 1],
                    padding='SAME')

second_layer += second_biases

second_layer = tf.nn.max_pool(value=second_layer,
                        ksize=[1, 2, 2, 1],
                        strides=[1, 2, 2, 1],
                        padding='SAME')
second_layer = tf.nn.relu(second_layer)

In [19]:
layer_flat, num_features = flatten_layer(second_layer)

In [20]:
fc_size = 128

In [21]:
fc1_weights = new_weights(shape=[num_features, fc_size])
fc1_biases = new_biases(length=fc_size)

fc1_layer = tf.matmul(layer_flat, fc1_weights) + fc1_biases

fc1_layer = tf.nn.relu(fc1_layer)

In [22]:
last_weights = new_weights(shape=[fc_size, num_classes])
last_biases = new_biases(length=num_classes)

last_layer = tf.matmul(fc1_layer, last_weights) + last_biases

In [23]:
y_pred = tf.nn.softmax(last_layer)
y_pred_cls = tf.argmax(y_pred, dimension=1)

In [24]:
cross_entropy = tf.nn.softmax_cross_entropy_with_logits(logits=last_layer,
                                                        labels=y_true)
cost = tf.reduce_mean(cross_entropy)

In [25]:
correct_prediction = tf.equal(y_pred_cls, y_true_cls)
accuracy = tf.reduce_mean(tf.cast(correct_prediction, tf.float32))

In [56]:
_1_weights_conv1 = pd.read_pickle('1st_conv\conv1_1_kernel_weights.csv').as_matrix()
_2_weights_conv1 = pd.read_pickle('1st_conv\conv1_2_kernel_weights.csv').as_matrix()
_3_weights_conv1 = pd.read_pickle('1st_conv\conv1_3_kernel_weights.csv').as_matrix()
_4_weights_conv1 = pd.read_pickle('1st_conv\conv1_4_kernel_weights.csv').as_matrix()
_5_weights_conv1 = pd.read_pickle('1st_conv\conv1_5_kernel_weights.csv').as_matrix()
_6_weights_conv1 = pd.read_pickle('1st_conv\conv1_6_kernel_weights.csv').as_matrix()
_1st_kernel = np.stack((_1_weights_conv1,
                         _2_weights_conv1,
                         _3_weights_conv1,
                         _4_weights_conv1,
                         _5_weights_conv1,
                         _6_weights_conv1), axis = 3)

_1st_biases = pd.read_csv('1st_conv_biases')
_1st_biases = _1st_biases.ix[:,'0']
_1st_biases = _1st_biases.as_matrix()
_1st_biases = np.reshape(_1st_biases, (6,))

In [58]:
_1_weights_conv2 = pd.read_pickle('2nd_conv\conv2_1_kernel_weights.csv').as_matrix()
_2_weights_conv2 = pd.read_pickle('2nd_conv\conv2_2_kernel_weights.csv').as_matrix()
_3_weights_conv2 = pd.read_pickle('2nd_conv\conv2_3_kernel_weights.csv').as_matrix()
_4_weights_conv2 = pd.read_pickle('2nd_conv\conv2_4_kernel_weights.csv').as_matrix()
_5_weights_conv2 = pd.read_pickle('2nd_conv\conv2_5_kernel_weights.csv').as_matrix()
_6_weights_conv2 = pd.read_pickle('2nd_conv\conv2_6_kernel_weights.csv').as_matrix()

_7_weights_conv2 = pd.read_pickle('2nd_conv\conv2_7_kernel_weights.csv').as_matrix()
_8_weights_conv2 = pd.read_pickle('2nd_conv\conv2_8_kernel_weights.csv').as_matrix()
_9_weights_conv2 = pd.read_pickle('2nd_conv\conv2_9_kernel_weights.csv').as_matrix()
_10_weights_conv2 = pd.read_pickle('2nd_conv\conv2_10_kernel_weights.csv').as_matrix()
_11_weights_conv2 = pd.read_pickle('2nd_conv\conv2_11_kernel_weights.csv').as_matrix()
_12_weights_conv2 = pd.read_pickle('2nd_conv\conv2_12_kernel_weights.csv').as_matrix()

_13_weights_conv2 = pd.read_pickle('2nd_conv\conv2_13_kernel_weights.csv').as_matrix()
_14_weights_conv2 = pd.read_pickle('2nd_conv\conv2_14_kernel_weights.csv').as_matrix()
_15_weights_conv2 = pd.read_pickle('2nd_conv\conv2_15_kernel_weights.csv').as_matrix()
_16_weights_conv2 = pd.read_pickle('2nd_conv\conv2_16_kernel_weights.csv').as_matrix()
_17_weights_conv2 = pd.read_pickle('2nd_conv\conv2_17_kernel_weights.csv').as_matrix()
_18_weights_conv2 = pd.read_pickle('2nd_conv\conv2_18_kernel_weights.csv').as_matrix()

_19_weights_conv2 = pd.read_pickle('2nd_conv\conv2_19_kernel_weights.csv').as_matrix()
_20_weights_conv2 = pd.read_pickle('2nd_conv\conv2_20_kernel_weights.csv').as_matrix()
_21_weights_conv2 = pd.read_pickle('2nd_conv\conv2_21_kernel_weights.csv').as_matrix()
_22_weights_conv2 = pd.read_pickle('2nd_conv\conv2_22_kernel_weights.csv').as_matrix()
_23_weights_conv2 = pd.read_pickle('2nd_conv\conv2_23_kernel_weights.csv').as_matrix()
_24_weights_conv2 = pd.read_pickle('2nd_conv\conv2_24_kernel_weights.csv').as_matrix()

_25_weights_conv2 = pd.read_pickle('2nd_conv\conv2_25_kernel_weights.csv').as_matrix()
_26_weights_conv2 = pd.read_pickle('2nd_conv\conv2_26_kernel_weights.csv').as_matrix()
_27_weights_conv2 = pd.read_pickle('2nd_conv\conv2_27_kernel_weights.csv').as_matrix()
_28_weights_conv2 = pd.read_pickle('2nd_conv\conv2_28_kernel_weights.csv').as_matrix()
_29_weights_conv2 = pd.read_pickle('2nd_conv\conv2_29_kernel_weights.csv').as_matrix()
_30_weights_conv2 = pd.read_pickle('2nd_conv\conv2_30_kernel_weights.csv').as_matrix()

_31_weights_conv2 = pd.read_pickle('2nd_conv\conv2_31_kernel_weights.csv').as_matrix()
_32_weights_conv2 = pd.read_pickle('2nd_conv\conv2_32_kernel_weights.csv').as_matrix()
_33_weights_conv2 = pd.read_pickle('2nd_conv\conv2_33_kernel_weights.csv').as_matrix()
_34_weights_conv2 = pd.read_pickle('2nd_conv\conv2_34_kernel_weights.csv').as_matrix()
_35_weights_conv2 = pd.read_pickle('2nd_conv\conv2_35_kernel_weights.csv').as_matrix()
_36_weights_conv2 = pd.read_pickle('2nd_conv\conv2_36_kernel_weights.csv').as_matrix()

_2nd_kernel = np.stack((_1_weights_conv2,
                         _2_weights_conv2,
                         _3_weights_conv2,
                         _4_weights_conv2,
                         _5_weights_conv2,
                         _6_weights_conv2,
                        
                         _7_weights_conv2,
                         _8_weights_conv2,
                         _9_weights_conv2,
                         _10_weights_conv2,
                         _11_weights_conv2,
                         _12_weights_conv2,
                        
                         _13_weights_conv2,
                         _14_weights_conv2,
                         _15_weights_conv2,
                         _16_weights_conv2,
                         _17_weights_conv2,
                         _18_weights_conv2,
                        
                         _19_weights_conv2,
                         _20_weights_conv2,
                         _21_weights_conv2,
                         _22_weights_conv2,
                         _23_weights_conv2,
                         _24_weights_conv2,
                        
                         _25_weights_conv2,
                         _26_weights_conv2,
                         _27_weights_conv2,
                         _28_weights_conv2,
                         _29_weights_conv2,
                         _30_weights_conv2,
                        
                         _31_weights_conv2,
                         _32_weights_conv2,
                         _33_weights_conv2,
                         _34_weights_conv2,
                         _35_weights_conv2,
                         _36_weights_conv2), axis = 3)

_2nd_biases = pd.read_csv('2nd_conv_biases')
_2nd_biases = _2nd_biases.ix[:,'0']
_2nd_biases = _2nd_biases.as_matrix()
_2nd_biases = np.reshape(_2nd_biases, (36,))

In [77]:
fc_1st_weights = pd.read_csv('1st_ff_weights')
fc_1st_weights = fc_1st_weights.ix[:,'0':]
fc_1st_weights = fc_1st_weights.as_matrix()

fc_1st_biases = pd.read_csv('1st_ff_biases')
fc_1st_biases = fc_1st_biases.ix[:,'0':]
fc_1st_biases = fc_1st_biases.as_matrix()
fc_1st_biases = np.reshape(fc_1st_biases, (128,))

In [87]:
fc_last_weights = pd.read_csv('last_ff_weights')
fc_last_weights = fc_last_weights.ix[:,'0':]
fc_last_weights = fc_last_weights.as_matrix()

fc_last_biases = pd.read_csv('last_ff_biases')
fc_last_biases = fc_last_biases.ix[:,'0':]
fc_last_biases = fc_last_biases.as_matrix()
fc_last_biases = np.reshape(fc_last_biases, (10,))

In [89]:
session = tf.Session()
session.run(tf.global_variables_initializer())

In [92]:
feed_label_validate = one_hot_encoding(label_validate)
validate_class = np.array([label.argmax() for label in feed_label_validate])
feed_dict_validate = {x: validate,
                      y_true: feed_label_validate,
                      y_true_cls: validate_class,
                      
                      first_weights:_1st_kernel,
                      first_biases:_1st_biases,
                     
                      second_weights:_2nd_kernel,
                      second_biases:_2nd_biases,
                      
                      fc1_weights:fc_1st_weights,
                      fc1_biases:fc_1st_biases,
                      
                      last_weights:fc_last_weights,
                      last_biases:fc_last_biases}

In [93]:
def print_accuracy(feed_dict_):
    # Use TensorFlow to compute the accuracy.
    acc = session.run(accuracy, feed_dict_)
    
    # Print the accuracy.
    #print("Accuracy on test-set: {0:.1%}".format(acc))
    return acc

In [94]:
print_accuracy(feed_dict_validate)

0.94091177